# Example on line integrals of a vector field

In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
from matplotlib import cm

from glob import glob

import pyrotor
from pyrotor.data_analysis import add_derivatives

## Define setting

### Define cost function

In [ ]:
# Here a trajectory is defined by (y_1, y_2, y_1', y_2')

# Model power F(y)^T y' with F(y) = (0, y_1)
q_force = np.array([[0,0,0,1],
                    [0,0,0,0],
                    [0,0,0,0],
                    [1,0,0,0]])
# Model L2-norm of y'
q_norm = np.array([[0,0,0,0],
                   [0,0,0,0],
                   [0,0,1,0],
                   [0,0,0,1]])
# Define hyperparameter
alpha = .35
# Final quadratic part
q = alpha * q_norm - q_force

# Linear part
w = np.array([0,0,0,0])

# Constant part
c = 0

quadratic_model = [c, w, q]

### Define initial and final states

In [ ]:
endpoints = {'x1': {'start': .111,
                    'end': .912,
                    'delta': 0.0001},
             'x2': {'start': .926,
                    'end': .211,
                    'delta': 0.0001}}

### Define independent variable (time)

In [ ]:
independent_variable = {'start': .1,
                        'end': .9,
                        'frequency': .01}
# Compute number of evaluation points
delta_time = independent_variable['end'] - independent_variable['start']
delta_time /= independent_variable['frequency']
independent_variable['points_nb'] = int(delta_time) + 1

### Define constraints

In [ ]:
# x1 > 0
def f1(data):
    x1 = data["x1"].values
    return x1

# x1 < 1
def f2(data):
    x1 = data["x1"].values
    return 1 - x1

# x2 > 0
def f3(data):
    x2 = data["x2"].values
    return x2

# x2 < 1
def f4(data):
    x2 = data["x2"].values
    return 1 - x2

constraints = [f1, f2, f3, f4]

### Define functional basis

In [ ]:
basis = 'bspline'
if basis == 'legendre':
    basis_features = {'x1': 4,
                      'x2': 6}
elif basis == 'bspline':
    basis_features = {'knots': [.33, .66],
                      'x1': 3,
                      'x2': 4}

### Import reference trajectories

In [ ]:
reference_trajectories = pyrotor.datasets.load_toy_dataset('example_1')

## Optimize

### Create PyRotor class

In [ ]:
pyrotor_class = pyrotor.Pyrotor(quadratic_model, 
                                reference_trajectories, 
                                endpoints, 
                                constraints, 
                                basis, 
                                basis_features, 
                                independent_variable, 
                                n_best_trajectory_to_use=10,
                                opti_factor=1,
                                derivative=True,
                                use_quadratic_programming=False,
                                verbose=True)

### Execute PyRotor solver

In [ ]:
pyrotor_class.compute_optimal_trajectory()

## Compute gains

### Define power

In [ ]:
def power(traj, q):
    """
    Compute power of a force modelled by q along a trajectory.
    """
    if 'cost' in traj.columns:
        traj = traj.drop(['cost'], axis=1, inplace=False)
    traj_deriv = add_derivatives([traj], pyrotor_class.basis_dimension)
    traj_deriv = traj_deriv[0].values
    p = np.dot(traj_deriv, q)
    p = (p * traj_deriv).sum(-1)
    
    return p

### Compute optimised power and deduce work gains

In [ ]:
# Compute optimised power
P_opt = power(pyrotor_class.trajectory, q_force)
# Deduce optimised work
W_opt = np.sum(P_opt)
# Compute reference works
W_ref = [np.sum(power(traj, q_force)) for traj in reference_trajectories]
# Deduce gains [%]
gains = pd.Series([(W_opt - W) / np.abs(W) * 100 for W in W_ref], name='Work gains [%]')
# Display gains
pd.options.display.float_format = "{:,.2f}".format
print(gains.describe())

## Plot

In [ ]:
# Define time axis
X = np.linspace(independent_variable['start'],
                independent_variable['end'],
                independent_variable['points_nb'])

# Model vector field
positions_x, positions_y = np.meshgrid(np.arange(0,1,.1), np.arange(0,1,.2))
arrays_y = positions_x
arrays_x = np.zeros_like(arrays_y)

# Get maximal values for power
P_min = np.min(P_opt)
P_max = np.max(P_opt)

# Plot
fig, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=(12, 20))
# Plot first variable with respect to time
ax1.plot(X, pyrotor_class.trajectory['x1'])
ax1.plot(X, pyrotor_class.reference_trajectories[0]['x1'], linestyle=':')
ax1.set_xlabel('$t$')
ax1.set_ylabel('$x_1(t)$')
# Plot second variable with respect to time
ax2.plot(X, pyrotor_class.trajectory['x2'])
ax2.plot(X, pyrotor_class.reference_trajectories[0]['x2'], linestyle=':')
ax2.set_xlabel('$t$')
ax2.set_ylabel('$x_2(t)$')
# Plot in (x_1, x_2) space
ax3.plot(pyrotor_class.trajectory['x1'], pyrotor_class.trajectory['x2'], c='b', alpha=.5, label='Optimised')
im = ax3.scatter(pyrotor_class.trajectory['x1'], pyrotor_class.trajectory['x2'], c=P_opt, vmin=P_min, vmax=P_max)
for trajectory in pyrotor_class.reference_trajectories:
    ax3.plot(trajectory['x1'], trajectory['x2'], linestyle=":", label='_nolegend_')
ax3.quiver(positions_x, positions_y, arrays_x, arrays_y, color='r', width=.004, alpha=.5)
fig.colorbar(im, ax=ax3)
ax3.set_xlabel('$x_1$')
ax3.set_ylabel('$x_2$')
ax3.set_xlim(left=0, right=1)
ax3.set_ylim(bottom=0, top=1)
ax3.legend()
plt.tight_layout()